# Logistic Regression Including REFINED Variables

### We will use the features from the XGBoost refined features model 
### This ensures consistency, reduces complexity, and leverages the feature selection process that has already been validated with my refined features XGBoost model
### This would also allow for a fair comparison between models

## Based on the table and plot of the XGBoost with all Features, an F score threshold of 30 is appropriate 
### We will exclude features that scored less than 30 in the refined features XGBoost training to retain only the most significant features and reduce overall complexity
## Features to Keep:
### 4   Ecliptic Latitude deg	271.0
### 2	Galactic Latitude deg	250.0
### 3	Galactic Longitude deg	223.0
### 5	Ecliptic Longitude deg	201.0
### 11	Discovery Year	132.0
### 1	Number of Planets	78.0
### 6	Number of Photometry Time Series	32.0

## Features to Exclude:
### 0	Number of Stars	20.0
### 7	Number of Radial Velocity Time Series	17.0
### 8	Number of Stellar Spectra Measurements	14.0
### 10	Controversial Flag	7.0
### 9	Circumbinary Flag	5.0

### As well as every other feature that didn't make it on the graph

#### After training and testing without number of stars, the test accuracy went down approximately 0.3%, (96.42% to 96.11%) 
#### Readding 'Number of Stars' to the training data, accuracy went back up, so we will keep 'Number of Stars' as a feature

In [1]:
import pandas as pd
composite_preprocessed = pd.read_csv('Composite_preprocessed_NO_MV_BALANCED.csv')
composite_preprocessed.head()

,Number of Stars,Number of Planets,Number of Moons,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Radial Velocity Time Series,Number of Stellar Spectra Measurements,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Circumbinary Flag,Controversial Flag,Discovery Year,Detected by Transits
0,3.094076,-0.666894,0.0,2.424559,1.234306,-0.145901,-0.624689,0.613405,1.731519,-0.261600,-0.012466,-0.044364,0,0,2007,0
1,-0.256668,-0.666894,0.0,1.186720,-0.283545,1.148661,-0.992906,0.613405,0.729626,-0.261600,-0.012466,-0.044364,0,0,2009,0
2,-0.256668,-0.666894,0.0,-0.877523,-0.306068,0.308947,-2.327878,0.613405,0.729626,-0.261600,-0.012466,-0.044364,0,0,2008,0
3,-0.256668,0.216988,0.0,1.382856,-0.669803,0.872499,-0.152934,0.613405,3.735304,0.591749,-0.012466,-0.044364,0,0,2002,0
4,6.444820,-0.666894,0.0,0.261241,-0.531444,1.023143,0.855489,0.613405,3.735304,2.298449,-0.012466,-0.044364,0,0,1996,0


# Drop Unnecessary

In [2]:
remove = ['Number of Radial Velocity Time Series',
          'Number of Stellar Spectra Measurements',
          'Controversial Flag',
          'Circumbinary Flag',
          'Number of Moons'
          ]
composite_refined = composite_preprocessed.drop(remove, axis=1)
composite_refined.head()

,Number of Stars,Number of Planets,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Discovery Year,Detected by Transits
0,3.094076,-0.666894,2.424559,1.234306,-0.145901,-0.624689,0.613405,-0.012466,-0.044364,2007,0
1,-0.256668,-0.666894,1.186720,-0.283545,1.148661,-0.992906,0.613405,-0.012466,-0.044364,2009,0
2,-0.256668,-0.666894,-0.877523,-0.306068,0.308947,-2.327878,0.613405,-0.012466,-0.044364,2008,0
3,-0.256668,0.216988,1.382856,-0.669803,0.872499,-0.152934,0.613405,-0.012466,-0.044364,2002,0
4,6.444820,-0.666894,0.261241,-0.531444,1.023143,0.855489,0.613405,-0.012466,-0.044364,1996,0


# Train Test Split

### Although we were able to filter out some variables from the model, we should still check VIF (Variance Inflation Factor) to ensure there is no multicollinearity in these remaining variables

### VIF measures how much the variance of a regression coefficient is inflated due to multicollinearity with other predictors
### A VIF value greater than 5 indicates high multicollinearity

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# we are trying to predict whether an exoplanet has been detected by transits
targets = composite_refined['Detected by Transits']
# training features are all variables except the targets
features = composite_refined.drop(['Detected by Transits'], axis=1)

### Add constant for VIF calculation

In [4]:
features_with_intercept = add_constant(features)

### Calculate VIF

In [5]:
vif_data = pd.DataFrame()
vif_data['Feature'] = features_with_intercept.columns
vif_data['VIF'] = [variance_inflation_factor(features_with_intercept.values, i) for i in range(features_with_intercept.shape[1])]
vif_data

,Feature,VIF
0,const,183017.944260
1,Number of Stars,1.042952
2,Number of Planets,1.038336
3,Galactic Latitude [deg],1.576786
4,Galactic Longitude [deg],2.207166
5,Ecliptic Latitude [deg],3.087281
6,Ecliptic Longitude [deg],1.356383
7,Number of Photometry Time Series,1.120718
8,Number of Emission Spectroscopy Measurements,1.004541
9,Number of Transmission Spectroscopy Measurements,1.007780


### VIF values of 4-5 are moderate. None of the features demonstrate moderate to high VIF

## Observe Feature Correlations

In [6]:
features_final=features
features_final.corr()

,Number of Stars,Number of Planets,Galactic Latitude [deg],Galactic Longitude [deg],Ecliptic Latitude [deg],Ecliptic Longitude [deg],Number of Photometry Time Series,Number of Emission Spectroscopy Measurements,Number of Transmission Spectroscopy Measurements,Discovery Year
Number of Stars,1.000000,0.109074,-0.044988,0.104236,-0.087528,-0.069450,0.057734,-0.003200,-0.002096,-0.147575
Number of Planets,0.109074,1.000000,-0.055511,0.008589,0.026680,-0.023926,-0.002505,0.057797,0.076409,-0.072274
Galactic Latitude [deg],-0.044988,-0.055511,1.000000,-0.019213,0.463063,0.174632,-0.090369,-0.025691,-0.008918,0.031820
Galactic Longitude [deg],0.104236,0.008589,-0.019213,1.000000,-0.657044,-0.408779,0.131333,-0.008283,-0.026088,-0.106558
Ecliptic Latitude [deg],-0.087528,0.026680,0.463063,-0.657044,1.000000,0.498442,-0.222283,-0.006864,0.025212,0.081113
Ecliptic Longitude [deg],-0.069450,-0.023926,0.174632,-0.408779,0.498442,1.000000,-0.119612,0.013811,0.031078,0.058041
Number of Photometry Time Series,0.057734,-0.002505,-0.090369,0.131333,-0.222283,-0.119612,1.000000,-0.004172,-0.014848,-0.252983
Number of Emission Spectroscopy Measurements,-0.003200,0.057797,-0.025691,-0.008283,-0.006864,0.013811,-0.004172,1.000000,-0.000553,0.000791
Number of Transmission Spectroscopy Measurements,-0.002096,0.076409,-0.008918,-0.026088,0.025212,0.031078,-0.014848,-0.000553,1.000000,-0.015425
Discovery Year,-0.147575,-0.072274,0.031820,-0.106558,0.081113,0.058041,-0.252983,0.000791,-0.015425,1.000000


## Weak correlation between these pairs of features:
### Ecliptic Latitude and Number of Photometry Time Series; -0.222
### Number of Photometry Time Series and Discovery Year; -0.253


## Moderate correlation between these pairs of feaures:
### Galactic Latitude and Ecliptic Latitude; .463
### Galactic Longitude and Ecliptic Latitude; -.657
### Galactic Longitude and Ecliptic Longitude; -.408
### Ecliptic Latitude and Ecliptic Longitude; .498


## The rest of the feature pairs can be characterized as having very weak correlations

# Random seed for reproducability

### Every time we use random_state=42 it will shuffle the same random way

In [7]:
import random
import numpy as np


def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)

# Train Test Split

## Split data

In [8]:
# Splitting dataset into training, testing and validation addresses overfitting
# shuffling is necessary to remove dependencies that come from order of data


# 80:20 split, 80% training and 20% testing 
# Since we are using GridSearch with cross validation, this handles the validation portion of our training 

# split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, targets, test_size = 0.2, random_state = 42, shuffle=True)

# Fitting the Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Hyperparameter Tuning
### Use Gridsearch with Cross-Validation to find optimal hyperparameters for the model while validating the model's performance
### Penalty: regularization l1 or l2
### C: list of possible values for the inverse of regularization strength 
#### Smaller values of C imply stronger regularization (to prevent overfitting), while larger values imply weaker regularization (allowing the model to fit the training data more closely)
### Solver: algorithm to use in the optimization problem (liblinear is suitable for small datasets and supports l1 and l2 regularization)
### Max iterations: This is to ensure our optimization algorithm has enough iterations to reach convergence 
####  Logregs especially with regularization require higher number of iterations to find optimal coefficients 
#### Avoids premature stopping 

## L1 regularization (Lasso):
### Enhanced feature selection (shrinks some features to exactly 0, simplifying our model with only the most important features)
### Handles multicollinearity ; penalizes large coefficients and encourages a sparse model
### Prevents overfitting ; Penalty for large coefficients reduces overfitting especially when number features is large compared to num obs
### Adds a penalty equal to the absolute value of the magnitude of coefficients

## L2 regularization (Ridge):
### Reduces overfitting by penalizing large coefficients; produces a more generalizable model
### Useful when dealing with datasets that have multicollinearity or when you want to retain all features but control their influence
### Adds a penalty equal to the square of the magnitude of coefficients

## Elasticnet:
### Combines both L1 and L2 regularization

## Liblinear Solver:
### Suitable for small-medium sized datasets (uses coordinate descent algorithm, efficient for L1 regularization
### Supports L1 regularization (for logreg with L1 penalty)
### Works great with binary classification problems

## LBFGS solver:
### Quasi-newton method widely used for optimizing logistic regression models
### Effective for binary classification problems, for small and large datasets
### Memory efficient (stores only a few vectors that represent the approximation to the Hessian matrix) great for large number of features
### Fast and stable convergence
### Worth considering as an alternative to Liblinear

### We will use Gridsearch Cross-Validation to find the optimal hyperparameters 

In [10]:
from sklearn.model_selection import GridSearchCV

logreg = LogisticRegression(random_state=42)

parameter_grid = {
    'penalty' : ['l1','l2'],
    'C' : [0.1, 1, 10, 100],
    'solver' : ['liblinear', 'lbfgs'],
    'max_iter':[500,1000,1500,2000],
}

grid_search = GridSearchCV(logreg, parameter_grid, cv=5, scoring = 'accuracy')
grid_search.fit(x_train, y_train)

/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kevin/anaconda3/envs/py310_ds/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
80 fits failed out of a total of 320.
The score on these train-test partitions for these pa

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
             param_grid={'C': [0.1, 1, 10, 100],
                         'max_iter': [500, 1000, 1500, 2000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'lbfgs']},
             scoring='accuracy')

# Extract best logistic regression model from Gridsearch CV

In [11]:
best_log_reg = grid_search.best_estimator_
print(f'Best hyper parameters: {grid_search.best_params_}')

Best hyper parameters: {'C': 0.1, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear'}


# Make predictions on the Test Set

In [12]:
y_pred_best_logreg = best_log_reg.predict(x_test)

# Best Logistic Regression Performance Metrics 

In [13]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix

cm = confusion_matrix(y_test, y_pred_best_logreg)
best_logreg_accuracy = accuracy_score(y_test, y_pred_best_logreg)
classif_report = classification_report(y_test, y_pred_best_logreg)
roc_score = roc_auc_score(y_test, y_pred_best_logreg)

print(
    cm,
    '\nLogistic Regressison Accuracy on Test Set:', best_logreg_accuracy,
    '\nLogistic Regressison Classification Report on Test Set:\n', classif_report,
    '\nLogistic Regressison ROC AUC Score on Test Set:\n', roc_score) 

[[550 101]
 [ 89 548]] 
Logistic Regressison Accuracy on Test Set: 0.8524844720496895 
Logistic Regressison Classification Report on Test Set:
               precision    recall  f1-score   support

           0       0.86      0.84      0.85       651
           1       0.84      0.86      0.85       637

    accuracy                           0.85      1288
   macro avg       0.85      0.85      0.85      1288
weighted avg       0.85      0.85      0.85      1288
 
Logistic Regressison ROC AUC Score on Test Set:
 0.8525683226144055


# Analyze Summary Table with coefficients (weights) and intercept (bias)

In [14]:
best_log_reg.intercept_, best_log_reg.coef_

(array([0.]),
 array([[-2.10578160e-01,  4.26254869e-01, -1.47981333e-01,
          3.47761828e-01,  1.69120681e+00,  4.24375698e-01,
         -4.21684847e+00,  0.00000000e+00,  8.56945894e-02,
         -3.98741413e-04]]))

### Match up coefs with features

In [15]:
feature_name = features_final.columns.values
summary_table = pd.DataFrame(columns=['Feature Name'], data = feature_name)

### Transpose bc by default np arrays are rows and not columns

In [16]:
summary_table['Coefficients'] = np.transpose(best_log_reg.coef_)

### Insert intercept as 0th index ; Move all indices up by 1 

In [17]:
summary_table.index += 1
summary_table.loc[0] = ['Intercept', best_log_reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature Name,Coefficients
0,Intercept,0.000000
1,Number of Stars,-0.210578
2,Number of Planets,0.426255
3,Galactic Latitude [deg],-0.147981
4,Galactic Longitude [deg],0.347762
5,Ecliptic Latitude [deg],1.691207
6,Ecliptic Longitude [deg],0.424376
7,Number of Photometry Time Series,-4.216848
8,Number of Emission Spectroscopy Measurements,0.000000
9,Number of Transmission Spectroscopy Measurements,0.085695


## The closer the coefficient is to 0, the less impact (less weight on the model) 
## Now we calculate log odds 
## Log odds tell us the strength of the relationship between the feature and the outcome (detected by transits T/F)

In [18]:
summary_table['Odds_ratio'] = np.exp(summary_table['Coefficients'])
summary_ordered = summary_table.sort_values('Odds_ratio', ascending=False)
summary_ordered

,Feature Name,Coefficients,Odds_ratio
5,Ecliptic Latitude [deg],1.691207,5.426025
2,Number of Planets,0.426255,1.531511
6,Ecliptic Longitude [deg],0.424376,1.528636
4,Galactic Longitude [deg],0.347762,1.415895
9,Number of Transmission Spectroscopy Measurements,0.085695,1.089474
0,Intercept,0.000000,1.000000
8,Number of Emission Spectroscopy Measurements,0.000000,1.000000
10,Discovery Year,-0.000399,0.999601
3,Galactic Latitude [deg],-0.147981,0.862447
1,Number of Stars,-0.210578,0.810116


## A feature is not important if:
## Coef ~ 0 (means this coefficient, even at its highest value will have close to 0 effect on the outcome);
## Odds ratio ~ 1 (means this coefficient, even at its highest value will only have minimal effect on the odds of the outcome)

## Concluding the Logistic Regression (sklearn) machine learning model
### Used XGBoost results for feature refinement and model simplification
### used VIF to identify and remove features with high multicollinearity to ensure model stability
### Split the data 80:20 training:testing with a set random seed (42)
### GridSearchCV found the optimal parameter values for penalty, regularization strength, solver and max iterations
### Model achieved mean cross-validation score of 0.8576, which indicates good overall performance 
### The confusion matrix and metrics show a balanced performance with high precision and recall for both classes
### [[550 101]
###  [ 89 548]]
### Precision class 1 = TP / TP + FP = 550 / 550+101 = 0.84
### Recall class 1 = TP / TP + FN = 550 / 550+89 = 0.86
### Precision class 0 = TN / TN + FN = 548 / 548+89 = 0.86
### Recall class 0 = TN / TN + FP = 548 / 548+101 = 0.84
### ROCAUC score on test set is 0.8525
### The good ROC-AUC score and balanced precision and recall metrics indicate that the model is effective at distinguishing between exoplanets discovered by transits and those discovered by other means
### The confusion matrix revealed that the model has a relatively low number of false positives and false negatives, highlighting the model's reliability
### According to the logistic regression, these variables had the most impact:
### Feature Name	Coefficients	Odds_ratio
### 5	Ecliptic Latitude [deg]	1.691207	5.426025
### 2	Number of Planets	0.426255	1.531511
### 6	Ecliptic Longitude [deg]	0.424376	1.528636
### 4	Galactic Longitude [deg]	0.347762	1.415895